In [1]:
import time
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# CONSTS

In [2]:
CHROME_DRIVER_PATH = "./chromedriver-linux/chromedriver"
BRAVE_PATH = "/usr/bin/brave-browser"
DOWNLOAD_DIRECTORY = "/home/mamoudou/dossier_partage/donnees-meteo/download"
ARCHIVE_DIRECTORY = "/home/mamoudou/dossier_partage/donnees-meteo/archive/daily"

In [3]:
now = datetime.datetime.now()

formatted_date = now.strftime("%Y%m%d %H").split(" ")
#DATE, HEURE = formatted_date[0], "{:02d}".format(int(formatted_date[1])-2)
DATE, HEURE = "20250201", "21"

print(f"Voici la date du jour recuperer: {formatted_date}.\nAprès répartiton, on a une date: {DATE} et une heure: {HEURE}")

URL = f"https://donneespubliques.meteofrance.fr/?fond=donnee_libre&prefixe=Txt%2FSynop%2Fsynop&extension=csv&date={DATE}&reseau={HEURE}"

print(f"L'URL formée après, devient: {URL}")

Voici la date du jour recuperer: ['20250202', '00'].
Après répartiton, on a une date: 20250201 et une heure: 21
L'URL formée après, devient: https://donneespubliques.meteofrance.fr/?fond=donnee_libre&prefixe=Txt%2FSynop%2Fsynop&extension=csv&date=20250201&reseau=21


## Browser Configs

In [4]:
options = Options()
options.binary_location = BRAVE_PATH  
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--nosandbox")
options.add_argument("--disable-dev-shm-usage")

prefs = {
    "download.default_directory": DOWNLOAD_DIRECTORY,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True 
}
options.add_experimental_option("prefs", prefs)

In [5]:
service = Service(executable_path=CHROME_DRIVER_PATH)

driver = webdriver.Chrome(service=service, options=options)

try:
    print(f"Téléchargement pour la date : {DATE}")
    driver.get(URL)
    time.sleep(5)
    print(f"Téléchargement terminé !!!")

except Exception as e:
    print(f"Erreur lors du téléchargement de {DATE}: {e}")

driver.close()
driver.quit()

Téléchargement pour la date : 20250201


Téléchargement terminé !!!


## Fixing the file extension problem

In [6]:
from file_management import change_file_extension

cvs_file = change_file_extension(DOWNLOAD_DIRECTORY)
print(f"The file I'm working with: {cvs_file}")

Le fichier a été renommé : /home/mamoudou/dossier_partage/donnees-meteo/download/synop.2025020121.txt -> /home/mamoudou/dossier_partage/donnees-meteo/download/synop.2025020121.csv
The file I'm working with: /home/mamoudou/dossier_partage/donnees-meteo/download/synop.2025020121.csv


# Starting the ELT

In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv(cvs_file, sep=";", dtype={"date": str})

print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd         ff           t          td    u     vv  ww  w1  w2    n nbas  hbas  cl  cm  ch    pres niv_bar geop tend24        tn12        tn24        tx12        tx24     tminsol  sw  tw      raf10     rafper  per etat_sol  ht_neige    ssfrai perssfrai        rr1       rr3       rr6       rr12       rr24 phenspe1 phenspe2 phenspe3 phenspe4 nnuage1 ctype1 hnuage1 nnuage2 ctype2 hnuage2 nnuage3 ctype3 hnuage3 nnuage4 ctype4 hnuage4  Unnamed: 59
0        7005  20250201210000  102820   -60        7   80   2.000000  273.950000  272.450000   90  18580   0  mq  mq   mq   mq    mq  mq  mq  mq  101900      mq   mq   -110          mq          mq          mq          mq  273.750000  mq  mq   3.000000   3.400000  -10        4  0.000000        mq        mq   0.000000  0.000000  0.000000   0.000000   0.000000       mq       mq       mq       mq      mq     mq      mq      mq     mq      mq      mq     mq      mq      mq     mq      mq          Na

# Récupération des informations importantes à savoir :
1. Indicatif OMM station : numéro de station `numer_sta` -> `int`
2. Date (UTC) : `date` -> `datetime`
3. Pression au niveau mer : `pmer` -> `int`
4. Variation de pression en 3 heures : `tend` -> `int`
5. Type de tendance barométrique : `cod_tend` -> `int`
6. Direction du vent moyen 10 mn : `dd` -> `int`
7. Vitesse du vent moyen 10 mn : `ff` -> `float`
8. Température : `t` -> `float`
9. Point de rosée : `td` -> `float`
10. Humidité : `u` -> `int`
11. Visibilité horizontale : `vv` -> `float`
12. Temps présent : `ww` -> `int`
13. Nébulosité totale : `n` -> `float`
14. Nébulosité des nuages de l'étage inférieur : `nbas` -> `int`
15. Hauteur de la base des nuages de l'étage inférieur : `hbas` -> `int`
16. Pression station : `pres` -> `int`
17. Variation de pression en 24 heures : `tend24` -> `int`
18. Température minimale sur N heures : `tn12` -> `float`
19. Température maximale sur N heures : `tx12` -> `float`
20. Température minimale du sol sur 12 heures : `tminsol` -> `float`
21. Rafales sur les 10 dernières minutes : `raf10` -> `float`
22. Rafales sur une période : `rafper` -> `float`
23. Période de mesure des rafales : `per` -> `float`
24. Précipitations dans les N dernières heures : `rr12` -> `float`

In [8]:
df = df[['numer_sta', 'date', "pmer", "tend", "cod_tend", "dd", "ff", "t", "td", "u", "vv", "ww", "n", "nbas", "hbas","pres", "tend24", "tn12", "tx12", "tminsol", "raf10", "rafper", "per", "rr12" ]]


print(df.to_string())


    numer_sta            date    pmer  tend cod_tend   dd         ff           t          td    u     vv  ww    n nbas  hbas    pres tend24        tn12        tx12     tminsol      raf10     rafper  per       rr12
0        7005  20250201210000  102820   -60        7   80   2.000000  273.950000  272.450000   90  18580   0   mq   mq    mq  101900   -110          mq          mq  273.750000   3.000000   3.400000  -10   0.000000
1        7015  20250201210000  102850   -80        6   80   2.700000  273.950000  271.850000   86   8490   0   mq    0    mq  102250   -200          mq          mq  272.850000   3.100000   4.000000  -10   0.200000
2        7020  20250201210000  102540  -150        8  120   4.200000  277.550000  275.850000   89     mq  mq   mq   mq    mq  102430   -190          mq          mq          mq   6.000000   6.000000  -10   0.400000
3        7027  20250201210000  102690  -100        8  120   2.600000  273.050000  272.650000   97   3590  10   mq    0    mq  101840     10     

## Remplacer les valeurs manquantes `mq` par `NaN`

In [9]:
df.replace("mq", np.nan, inplace=True)
print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd         ff           t          td    u     vv   ww    n nbas  hbas    pres tend24        tn12        tx12     tminsol      raf10     rafper  per       rr12
0        7005  20250201210000  102820   -60        7   80   2.000000  273.950000  272.450000   90  18580    0  NaN  NaN   NaN  101900   -110         NaN         NaN  273.750000   3.000000   3.400000  -10   0.000000
1        7015  20250201210000  102850   -80        6   80   2.700000  273.950000  271.850000   86   8490    0  NaN    0   NaN  102250   -200         NaN         NaN  272.850000   3.100000   4.000000  -10   0.200000
2        7020  20250201210000  102540  -150        8  120   4.200000  277.550000  275.850000   89    NaN  NaN  NaN  NaN   NaN  102430   -190         NaN         NaN         NaN   6.000000   6.000000  -10   0.400000
3        7027  20250201210000  102690  -100        8  120   2.600000  273.050000  272.650000   97   3590   10  NaN    0   NaN  101840     10

## Nettoyages de colonnes non renseignées 

1. Informations générales
	-	Numéro de station : `numer_sta` -> int
	-	Date (UTC) : `date` -> datetime
2. Pression
	-	Pression au niveau mer : `pmer` -> int
	-	Variation de pression en 3 heures : `tend` -> int
	-	Type de tendance barométrique : `cod_tend` -> int
	-	Pression station : `pres` -> int
	-	Variation de pression en 24 heures : `tend24` -> int
3. Vent
	-	Direction du vent moyen 10 mn : `dd` -> int
	-	Vitesse du vent moyen 10 mn : `ff` -> float
	-	Rafales sur les 10 dernières minutes : `raf10` -> float
	-	Rafales sur une période : `rafper` -> float
	-	Période de mesure des rafales : `per` -> float
4. Température
	-	Température : `t` -> float
	-	Point de rosée : `td` -> float
	-	Température minimale sur N heures : `tn12` -> float
	-	Température maximale sur N heures : `tx12` -> float
	-	Température minimale du sol sur 12 heures : `tminsol` -> float
5. Humidité
	-	Humidité : `u` -> int
6. Visibilité
	-	Visibilité horizontale : `vv` -> float
7. Nébulosité et nuages
	-	Nébulosité totale : `n` -> float
	-	Nébulosité des nuages de l’étage inférieur : `nbas` -> int
	-	Hauteur de la base des nuages de l’étage inférieur : `hbas` -> int
8. Précipitations
	-	Précipitations dans les N dernières heures : `rr12` -> float
9. Temps présent
	-	Temps présent : `ww` -> int

In [10]:
df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]] = df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]].fillna(0)

moyenne = ["pmer", "tend", "cod_tend", "pres", "tend24", "t", "td", "tn12", "tx12", "tminsol", "u", "vv", "n", "nbas", "hbas", "ww"]
df[moyenne] = df[moyenne].apply(pd.to_numeric, errors='coerce')

for col in moyenne:
    df[col] = df[col].fillna(df[col].mean())

print(df.to_string())



    numer_sta            date           pmer        tend  cod_tend   dd         ff          t          td           u            vv         ww           n      nbas         hbas           pres      tend24    tn12    tx12     tminsol      raf10     rafper  per       rr12
0        7005  20250201210000  102820.000000  -60.000000   7.00000   80   2.000000  273.95000  272.450000   90.000000  18580.000000   0.000000   83.333333  3.076923  1127.894737  101900.000000  -110.00000  268.55  273.35  273.750000   3.000000   3.400000  -10   0.000000
1        7015  20250201210000  102850.000000  -80.000000   6.00000   80   2.700000  273.95000  271.850000   86.000000   8490.000000   0.000000   83.333333  0.000000  1127.894737  102250.000000  -200.00000  268.55  273.35  272.850000   3.100000   4.000000  -10   0.200000
2        7020  20250201210000  102540.000000 -150.000000   8.00000  120   4.200000  277.55000  275.850000   89.000000  20675.555556   2.909091   83.333333  3.076923  1127.894737  102430.0

## Typages des colonnes

In [11]:
df["date"] = pd.to_datetime(df["date"], format='%Y%m%d%H%M%S')
df['date'] = df['date'].dt.strftime("%Y-%m-%d %H:%M:%S")

entier = ["numer_sta", "pmer", "tend", "cod_tend", "dd", "u", "ww", "nbas", "hbas", "pres", "tend24"]
for i in entier:
    print(f"{i} -> {df[i].dtype}")
    df[i] = df[i].astype(int)
    print(f"{i} -> {df[i].dtype}")
    print("")

double = ['dd', 'ff', 'raf10', 'rafper', 'rr12', "per", "t", "td", "tn12", "tx12", "tminsol", "n", "vv"]
for i in double:
    print(f"{i} -> {df[i].dtype}")
    df[i] = df[i].astype(float)
    print(f"{i} -> {df[i].dtype}")
    print("")

numer_sta -> int64
numer_sta -> int64

pmer -> float64
pmer -> int64

tend -> float64
tend -> int64

cod_tend -> float64
cod_tend -> int64

dd -> object
dd -> int64

u -> float64
u -> int64

ww -> float64
ww -> int64

nbas -> float64
nbas -> int64

hbas -> float64
hbas -> int64

pres -> float64
pres -> int64

tend24 -> float64
tend24 -> int64

dd -> int64
dd -> float64

ff -> object
ff -> float64

raf10 -> object
raf10 -> float64

rafper -> object
rafper -> float64

rr12 -> object
rr12 -> float64

per -> object
per -> float64

t -> float64
t -> float64

td -> float64
td -> float64

tn12 -> float64
tn12 -> float64

tx12 -> float64
tx12 -> float64

tminsol -> float64
tminsol -> float64

n -> float64
n -> float64

vv -> float64
vv -> float64



In [12]:
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff          t          td    u            vv  ww           n  nbas  hbas    pres  tend24    tn12    tx12     tminsol  raf10  rafper   per  rr12
0        7005  2025-02-01 21:00:00  102820   -60         7   80.0   2.0  273.95000  272.450000   90  18580.000000   0   83.333333     3  1127  101900    -110  268.55  273.35  273.750000    3.0     3.4 -10.0   0.0
1        7015  2025-02-01 21:00:00  102850   -80         6   80.0   2.7  273.95000  271.850000   86   8490.000000   0   83.333333     0  1127  102250    -200  268.55  273.35  272.850000    3.1     4.0 -10.0   0.2
2        7020  2025-02-01 21:00:00  102540  -150         8  120.0   4.2  277.55000  275.850000   89  20675.555556   2   83.333333     3  1127  102430    -190  268.55  273.35  276.377778    6.0     6.0 -10.0   0.4
3        7027  2025-02-01 21:00:00  102690  -100         8  120.0   2.6  273.05000  272.650000   97   3590.000000  10   83.333333     0  1127  10184

## Conversion des températures de Kelvin en Celcuis

In [13]:
celcuis = ["t", "td", "tn12", "tx12", "tminsol"]
for i in celcuis:
    df[i] = df[i] - 273.15

print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff         t         td    u            vv  ww           n  nbas  hbas    pres  tend24  tn12  tx12    tminsol  raf10  rafper   per  rr12
0        7005  2025-02-01 21:00:00  102820   -60         7   80.0   2.0   0.80000  -0.700000   90  18580.000000   0   83.333333     3  1127  101900    -110  -4.6   0.2   0.600000    3.0     3.4 -10.0   0.0
1        7015  2025-02-01 21:00:00  102850   -80         6   80.0   2.7   0.80000  -1.300000   86   8490.000000   0   83.333333     0  1127  102250    -200  -4.6   0.2  -0.300000    3.1     4.0 -10.0   0.2
2        7020  2025-02-01 21:00:00  102540  -150         8  120.0   4.2   4.40000   2.700000   89  20675.555556   2   83.333333     3  1127  102430    -190  -4.6   0.2   3.227778    6.0     6.0 -10.0   0.4
3        7027  2025-02-01 21:00:00  102690  -100         8  120.0   2.6  -0.10000  -0.500000   97   3590.000000  10   83.333333     0  1127  101840      10  -4.6   0.2   4.4000

## Arrondir tous les decimaux à 2chiffres après la virgules

In [14]:
df[double] = df[double].round(2)
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff      t     td    u        vv  ww       n  nbas  hbas    pres  tend24  tn12  tx12  tminsol  raf10  rafper   per  rr12
0        7005  2025-02-01 21:00:00  102820   -60         7   80.0   2.0   0.80  -0.70   90  18580.00   0   83.33     3  1127  101900    -110  -4.6   0.2     0.60    3.0     3.4 -10.0   0.0
1        7015  2025-02-01 21:00:00  102850   -80         6   80.0   2.7   0.80  -1.30   86   8490.00   0   83.33     0  1127  102250    -200  -4.6   0.2    -0.30    3.1     4.0 -10.0   0.2
2        7020  2025-02-01 21:00:00  102540  -150         8  120.0   4.2   4.40   2.70   89  20675.56   2   83.33     3  1127  102430    -190  -4.6   0.2     3.23    6.0     6.0 -10.0   0.4
3        7027  2025-02-01 21:00:00  102690  -100         8  120.0   2.6  -0.10  -0.50   97   3590.00  10   83.33     0  1127  101840      10  -4.6   0.2     4.40    3.6     3.7 -10.0   0.0
4        7037  2025-02-01 21:00:00  102780   -50       

# Add verif to check is all values

In [15]:
verif = ["pmer", "tend", "cod_tend", "pres", "tend24", "t", "td", "tn12", "tx12", "tminsol", "u", "vv", "n", "nbas", "hbas", "ww"]

for col in verif:
    if df[col].isna().all() :
        df[col].fillna(0.0,inplace=True)

print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff      t     td    u        vv  ww       n  nbas  hbas    pres  tend24  tn12  tx12  tminsol  raf10  rafper   per  rr12
0        7005  2025-02-01 21:00:00  102820   -60         7   80.0   2.0   0.80  -0.70   90  18580.00   0   83.33     3  1127  101900    -110  -4.6   0.2     0.60    3.0     3.4 -10.0   0.0
1        7015  2025-02-01 21:00:00  102850   -80         6   80.0   2.7   0.80  -1.30   86   8490.00   0   83.33     0  1127  102250    -200  -4.6   0.2    -0.30    3.1     4.0 -10.0   0.2
2        7020  2025-02-01 21:00:00  102540  -150         8  120.0   4.2   4.40   2.70   89  20675.56   2   83.33     3  1127  102430    -190  -4.6   0.2     3.23    6.0     6.0 -10.0   0.4
3        7027  2025-02-01 21:00:00  102690  -100         8  120.0   2.6  -0.10  -0.50   97   3590.00  10   83.33     0  1127  101840      10  -4.6   0.2     4.40    3.6     3.7 -10.0   0.0
4        7037  2025-02-01 21:00:00  102780   -50       

# Archive file

In [16]:
from file_management import move_file

move_file(cvs_file, ARCHIVE_DIRECTORY)

Le fichier a été déplacé de /home/mamoudou/dossier_partage/donnees-meteo/download/synop.2025020121.csv à /home/mamoudou/dossier_partage/donnees-meteo/archive/daily/synop.2025020121.csv


# Sending data to API

In [17]:
import requests
from api import send_data

In [18]:
API_URL = "http://127.0.0.1:8080/api/donnees-climatiques" 

dict_data = df.to_dict(orient="records")

for row in dict_data:
    print(row)
    send_data(API_URL, row, requests)

{'numer_sta': 7005, 'date': '2025-02-01 21:00:00', 'pmer': 102820, 'tend': -60, 'cod_tend': 7, 'dd': 80.0, 'ff': 2.0, 't': 0.8, 'td': -0.7, 'u': 90, 'vv': 18580.0, 'ww': 0, 'n': 83.33, 'nbas': 3, 'hbas': 1127, 'pres': 101900, 'tend24': -110, 'tn12': -4.6, 'tx12': 0.2, 'tminsol': 0.6, 'raf10': 3.0, 'rafper': 3.4, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7015, 'date': '2025-02-01 21:00:00', 'pmer': 102850, 'tend': -80, 'cod_tend': 6, 'dd': 80.0, 'ff': 2.7, 't': 0.8, 'td': -1.3, 'u': 86, 'vv': 8490.0, 'ww': 0, 'n': 83.33, 'nbas': 0, 'hbas': 1127, 'pres': 102250, 'tend24': -200, 'tn12': -4.6, 'tx12': 0.2, 'tminsol': -0.3, 'raf10': 3.1, 'rafper': 4.0, 'per': -10.0, 'rr12': 0.2}
Données ajoutées avec succès!
{'numer_sta': 7020, 'date': '2025-02-01 21:00:00', 'pmer': 102540, 'tend': -150, 'cod_tend': 8, 'dd': 120.0, 'ff': 4.2, 't': 4.4, 'td': 2.7, 'u': 89, 'vv': 20675.56, 'ww': 2, 'n': 83.33, 'nbas': 3, 'hbas': 1127, 'pres': 102430, 'tend24': -190, 'tn12': -4.6, 

Données ajoutées avec succès!
{'numer_sta': 7027, 'date': '2025-02-01 21:00:00', 'pmer': 102690, 'tend': -100, 'cod_tend': 8, 'dd': 120.0, 'ff': 2.6, 't': -0.1, 'td': -0.5, 'u': 97, 'vv': 3590.0, 'ww': 10, 'n': 83.33, 'nbas': 0, 'hbas': 1127, 'pres': 101840, 'tend24': 10, 'tn12': -4.6, 'tx12': 0.2, 'tminsol': 4.4, 'raf10': 3.6, 'rafper': 3.7, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7037, 'date': '2025-02-01 21:00:00', 'pmer': 102780, 'tend': -50, 'cod_tend': 8, 'dd': 60.0, 'ff': 2.3, 't': 0.2, 'td': -1.1, 'u': 91, 'vv': 14880.0, 'ww': 0, 'n': 83.33, 'nbas': 0, 'hbas': 1127, 'pres': 100790, 'tend24': -90, 'tn12': -4.6, 'tx12': 0.2, 'tminsol': 1.9, 'raf10': 2.9, 'rafper': 2.9, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7072, 'date': '2025-02-01 21:00:00', 'pmer': 102810, 'tend': -50, 'cod_tend': 8, 'dd': 110.0, 'ff': 1.8, 't': -1.3, 'td': -2.6, 'u': 91, 'vv': 20000.0, 'ww': 0, 'n': 83.33, 'nbas': 0, 'hbas': 1127, 'pres': 101590, 

Données ajoutées avec succès!
{'numer_sta': 7190, 'date': '2025-02-01 21:00:00', 'pmer': 102800, 'tend': -70, 'cod_tend': 8, 'dd': 0.0, 'ff': 0.0, 't': -0.2, 'td': -1.8, 'u': 89, 'vv': 15650.0, 'ww': 0, 'n': 83.33, 'nbas': 0, 'hbas': 1127, 'pres': 100880, 'tend24': -300, 'tn12': -4.6, 'tx12': 0.2, 'tminsol': -2.2, 'raf10': 0.5, 'rafper': 2.9, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7207, 'date': '2025-02-01 21:00:00', 'pmer': 101961, 'tend': -21, 'cod_tend': 5, 'dd': 150.0, 'ff': 6.2, 't': 6.8, 'td': 5.56, 'u': 83, 'vv': 15000.0, 'ww': 2, 'n': 100.0, 'nbas': 8, 'hbas': 1250, 'pres': 100327, 'tend24': -232, 'tn12': -4.6, 'tx12': 0.2, 'tminsol': 3.23, 'raf10': 9.4, 'rafper': 9.4, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7222, 'date': '2025-02-01 21:00:00', 'pmer': 102560, 'tend': -120, 'cod_tend': 8, 'dd': 80.0, 'ff': 3.3, 't': 1.0, 'td': -0.2, 'u': 92, 'vv': 11210.0, 'ww': 0, 'n': 83.33, 'nbas': 0, 'hbas': 1127, 'pres': 102230

Données ajoutées avec succès!
{'numer_sta': 7471, 'date': '2025-02-01 21:00:00', 'pmer': 101961, 'tend': -20, 'cod_tend': 5, 'dd': 50.0, 'ff': 1.1, 't': -0.5, 'td': -1.2, 'u': 95, 'vv': 8340.0, 'ww': 0, 'n': 100.0, 'nbas': 8, 'hbas': 150, 'pres': 92580, 'tend24': -260, 'tn12': -4.6, 'tx12': 0.2, 'tminsol': -0.3, 'raf10': 2.1, 'rafper': 2.6, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7481, 'date': '2025-02-01 21:00:00', 'pmer': 102620, 'tend': -20, 'cod_tend': 7, 'dd': 310.0, 'ff': 2.1, 't': 3.8, 'td': 0.5, 'u': 79, 'vv': 25000.0, 'ww': 2, 'n': 100.0, 'nbas': 8, 'hbas': 800, 'pres': 99640, 'tend24': -200, 'tn12': -4.6, 'tx12': 0.2, 'tminsol': 3.23, 'raf10': 3.3, 'rafper': 4.2, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7510, 'date': '2025-02-01 21:00:00', 'pmer': 102500, 'tend': -10, 'cod_tend': 6, 'dd': 260.0, 'ff': 0.8, 't': 2.5, 'td': 1.6, 'u': 94, 'vv': 7240.0, 'ww': 0, 'n': 83.33, 'nbas': 0, 'hbas': 1127, 'pres': 101890, 'tend

Données ajoutées avec succès!
{'numer_sta': 7661, 'date': '2025-02-01 21:00:00', 'pmer': 101961, 'tend': -21, 'cod_tend': 5, 'dd': 50.0, 'ff': 8.3, 't': 8.38, 'td': 5.56, 'u': 83, 'vv': 20675.56, 'ww': 2, 'n': 83.33, 'nbas': 3, 'hbas': 1127, 'pres': 100327, 'tend24': -232, 'tn12': -4.6, 'tx12': 0.2, 'tminsol': 3.23, 'raf10': 13.3, 'rafper': 13.9, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7690, 'date': '2025-02-01 21:00:00', 'pmer': 102160, 'tend': 20, 'cod_tend': 0, 'dd': 320.0, 'ff': 3.0, 't': 9.9, 'td': 8.0, 'u': 88, 'vv': 20000.0, 'ww': 2, 'n': 100.0, 'nbas': 8, 'hbas': 1127, 'pres': 101840, 'tend24': -80, 'tn12': -4.6, 'tx12': 0.2, 'tminsol': 9.2, 'raf10': 3.9, 'rafper': 5.4, 'per': -10.0, 'rr12': 1.6}


Données ajoutées avec succès!
{'numer_sta': 7747, 'date': '2025-02-01 21:00:00', 'pmer': 102300, 'tend': 30, 'cod_tend': 1, 'dd': 310.0, 'ff': 4.2, 't': 7.8, 'td': 3.3, 'u': 73, 'vv': 60000.0, 'ww': 0, 'n': 100.0, 'nbas': 8, 'hbas': 1250, 'pres': 101760, 'tend24': -430, 'tn12': -4.6, 'tx12': 0.2, 'tminsol': 7.0, 'raf10': 6.8, 'rafper': 7.3, 'per': -10.0, 'rr12': 0.8}
Données ajoutées avec succès!
{'numer_sta': 7761, 'date': '2025-02-01 21:00:00', 'pmer': 101930, 'tend': -50, 'cod_tend': 8, 'dd': 40.0, 'ff': 1.7, 't': 9.4, 'td': 8.3, 'u': 93, 'vv': 14770.0, 'ww': 0, 'n': 100.0, 'nbas': 8, 'hbas': 3300, 'pres': 101820, 'tend24': -410, 'tn12': -4.6, 'tx12': 0.2, 'tminsol': 8.5, 'raf10': 2.3, 'rafper': 4.0, 'per': -10.0, 'rr12': 0.6}
Données ajoutées avec succès!
{'numer_sta': 7790, 'date': '2025-02-01 21:00:00', 'pmer': 102120, 'tend': 20, 'cod_tend': 0, 'dd': 290.0, 'ff': 1.6, 't': 10.8, 'td': 10.2, 'u': 96, 'vv': 19570.0, 'ww': 0, 'n': 100.0, 'nbas': 7, 'hbas': 450, 'pres': 101970, 'ten